In [4]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow_hub as hub
import tensorflow as tf

In [5]:
df = pd.read_csv("train.csv")
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,VIDEO_NAME,CLASS
0,1,Brandon Pryor,2014-01-19 00:36:25,I dont even watch it anymore i just come here ...,PSY - GANGNAM STYLE(?????) M/V,0
1,2,Chelsea Yun,2015-05-23 07:17:09.691,i hate rap﻿,Eminem - Love The Way You Lie ft. Rihanna,0
2,3,Sofia Aristizabal,2014-09-09 00:43:52,"I loved, she is amazing.. OMG your eyes*_*﻿",Katy Perry - Roar,0
3,4,said abdesalam,2015-05-24 07:35:13.754,song is bad﻿,Eminem - Love The Way You Lie ft. Rihanna,0
4,5,crazy girl,2015-05-23 23:26:05.305,tension⤴︎⤴︎﻿,"LMFAO - Party Rock Anthem ft. Lauren Bennett, ...",0


Now, let's use the bert classifier. Just doing some testing first to get an  understanding 

In [7]:
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")


encoded_X = tokenizer.encode("What a beautiful red rose ")
print(encoded_X)

[101, 2054, 1037, 3376, 2417, 3123, 102]


In [8]:
import torch
from transformers import BertModel
from sklearn.metrics.pairwise import cosine_similarity

model = BertModel.from_pretrained("bert-base-uncased")


king_token_id = tokenizer.convert_tokens_to_ids(["king"])[0]
king_embedding = model.embeddings.word_embeddings(torch.tensor([king_token_id]))

queen_token_id = tokenizer.convert_tokens_to_ids(["queen"])[0]
queen_embedding = model.embeddings.word_embeddings(torch.tensor([queen_token_id]))

cos = torch.nn.CosineSimilarity(dim=1)
similarity = cos(king_embedding, queen_embedding)
print(similarity[0])

tensor(0.6469, grad_fn=<SelectBackward0>)


The code above is some basic experimentation I did. Now let's actually process the data 

In [10]:
df_spam = df[df["CLASS"] == 1]
df_ham = df[df["CLASS"] == 0]
df_spam.shape
df_ham.shape

(659, 6)

710 spams, 659 non-spam emails. We want equal amount of spam and ham this time, so reduce spam to 659

In [12]:
df_spam_downsampled = df_spam.sample(df_ham.shape[0])
df_spam_downsampled.shape

(659, 6)

In [13]:
df2 = pd.concat([df_spam_downsampled, df_ham])
df2.shape

(1318, 6)

In [14]:
df2.sample(5)

,COMMENT_ID,AUTHOR,DATE,CONTENT,VIDEO_NAME,CLASS
137,138,Louis Bryant,NaN,You guys should check out this EXTRAORDINARY w...,Eminem - Love The Way You Lie ft. Rihanna,1
1310,1311,Wafence,2013-09-04 19:16:44.109,"Hey Youtubers and All Music lover&#39;s, Guess...",Shakira - Waka Waka,1
1301,1302,maddog1431,2014-08-12 18:38:42,Check out my covers I have a video coming out ...,Katy Perry - Roar,1
409,410,Yeung Marvin,2014-11-12 23:56:54,Haha its so funny to see the salt of westerner...,PSY - GANGNAM STYLE(?????) M/V,0
1352,1353,Jay Stevens,NaN,COME SUBSCRIBE TO MY CHANNEL! ;-) PLEASE!!,Eminem - Love The Way You Lie ft. Rihanna,1


In [15]:
X = df2["CONTENT"]
Y = df2["CLASS"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, random_state = 3)

In [16]:
def bert_preprocess(texts, tokenizer, max_length=128):
    """
    Tokenize input texts for BERT with padding, truncation, and attention masks.
    """
    encoded = tokenizer(
        texts.tolist(),          # Convert Pandas Series to list
        padding=True,            # Pad to the longest sequence
        truncation=True,         # Truncate to max_length
        max_length=max_length,   # Set maximum sequence length
        return_tensors="tf"      # Return TensorFlow tensors
    )
    return encoded

# 4. Custom Encoding Function
def bert_encode(encoded_inputs, bert_model):
    """
    Extract pooled output ([CLS] token embedding) from BERT.
    """
    outputs = bert_model(encoded_inputs)
    pooled_output = outputs.pooler_output  # Shape: (batch_size, hidden_size)
    return pooled_output

In [17]:
import tensorflow_hub as hub
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2",
    trainable=True)


RuntimeError: Op type not registered 'CaseFoldUTF8' in binary running on users-MBP-5. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.